In [1]:
import pandas as pd
from tqdm import tqdm
from scipy import stats
import numpy as np

In [5]:
def backtest_code(code, benchmark = 2):
    rolling = 5
    benchmark_index = "US100"
    
    pivoted = res[[code, benchmark_index]]
    pivoted.ffill()
    ret = pivoted.pct_change(fill_method=None)
    try:
        ret['stock_price'] = pivoted[code]
        ret['ret_diff'] = ret[code] - ret[benchmark_index]
        # type 1
        # ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std() * 100
        # ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean() * 100
        # ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])/ret['ret_diff_std'])
        
        # type 2
        ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std()
        ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean()
        ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])*ret['ret_diff_std'])*10000

        ret[code] = (1 + ret[code]).cumprod() - 1

        ret['index'] = [i for i in range(len(ret))]

        # check direction
        ret['lag10'] = ret['stock_price'].shift(-4)
        ret['change_stock'] = (ret['lag10'] - ret[code])/ret[code]
        ret['direction'] = ret.apply(lambda x: -1 if x['stock_price'] - x['lag10'] > 0 else 1, axis=1)

        # check if the direction are the same, if not, tned to not being a mean reversion
        ret['lag10_twii'] = ret[benchmark_index].shift(-4)
        ret['change_twii'] = (ret['lag10_twii'] - ret[benchmark_index])/ret[benchmark_index]
        ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
        ret['filter_2'] = ret.apply(lambda x: 1 if abs(x['change_stock']) < abs(x['change_twii']) else 1, axis=1)

        # check raise
        ret[f'{code}_lag60'] = ret['stock_price'].shift(-12)
        ret['target_60mins'] = (ret[f'{code}_lag60'] - ret['stock_price'])/ret['stock_price'] * ret['direction']
        ret = ret[(ret['stdize_ret_diff'] >= benchmark) & (ret['filter_1'] == 1) & (ret['filter_2'] == 1)][['target_60mins', 'stdize_ret_diff']].dropna()
        ret['code'] = code
        return ret
    except:
        print(pivoted)

In [6]:
res = pd.read_parquet("US100.parquet")
code_list=list(set(res.columns))

dfs = []
for code in tqdm(code_list[:]):
    if code == 'US100': continue
    df = backtest_code(code)
    if len(df) == 0: continue
    dfs.append(df)

100%|██████████| 102/102 [00:04<00:00, 21.02it/s]


In [7]:
final = pd.concat(dfs)
final

code,target_60mins,stdize_ret_diff,code
datetime,,,
2024-07-23 09:40:00-04:00,0.016146,2.017278,CMCSA
2024-07-17 09:30:00-04:00,-0.000607,2.759601,KDP
2024-07-25 09:30:00-04:00,0.002664,3.399127,KDP
2024-07-01 09:30:00-04:00,-0.006549,4.838674,ORLY
2024-07-01 09:40:00-04:00,-0.000148,2.269441,ORLY
...,...,...,...
2024-08-06 09:30:00-04:00,0.009836,2.647611,MU
2024-08-14 09:30:00-04:00,0.017788,2.496968,MU
2024-08-23 09:30:00-04:00,0.019274,2.661238,MU


In [20]:
import numpy as np
import pandas as pd
from scipy import stats
# to test if return after 60 mins are greater than {test}
for test in range(0, 15, 1):
    test /= 1000
    final = pd.concat(dfs)
    final = final[final['target_60mins'] != 0.0]
    final['target_60mins'] = final['target_60mins'] - test
    mean = final['target_60mins'].mean()
    n = len(final)
    s = final['target_60mins'].std()

    t_statistic = (mean - 0.0) / (s / np.sqrt(n))

    t_stat, p_value = stats.ttest_1samp(final['target_60mins'], 0.0)

    if t_stat > 0:
        one_tailed_p_value = p_value / 2
    else:
        one_tailed_p_value = 1 - (p_value / 2)

    print(f"{test} P-value: {np.round(one_tailed_p_value, 2)} {np.round(t_statistic, 2)}")
print("US100", len(final))


0.0 P-value: 0.0 14.6
0.001 P-value: 0.0 13.35
0.002 P-value: 0.0 12.09
0.003 P-value: 0.0 10.83
0.004 P-value: 0.0 9.58
0.005 P-value: 0.0 8.32
0.006 P-value: 0.0 7.06
0.007 P-value: 0.0 5.81
0.008 P-value: 0.0 4.55
0.009 P-value: 0.0 3.29
0.01 P-value: 0.02 2.04
0.011 P-value: 0.22 0.78
0.012 P-value: 0.68 -0.47
0.013 P-value: 0.96 -1.73
0.014 P-value: 1.0 -2.99
US100 418


In [21]:
res = pd.read_parquet("US500.parquet")
code_list=list(set(res.columns))

dfs = []
for code in tqdm(code_list[:]):
    if code == 'US100': continue
    df = backtest_code(code)
    if len(df) == 0: continue
    dfs.append(df)

100%|██████████| 502/502 [01:49<00:00,  4.57it/s]


In [22]:
import numpy as np
import pandas as pd
from scipy import stats
# to test if return after 60 mins are greater than {test}
for test in range(0, 15, 1):
    test /= 1000
    final = pd.concat(dfs)
    final = final[final['target_60mins'] != 0.0]
    final['target_60mins'] = final['target_60mins'] - test
    mean = final['target_60mins'].mean()
    n = len(final)
    s = final['target_60mins'].std()

    t_statistic = (mean - 0.0) / (s / np.sqrt(n))

    t_stat, p_value = stats.ttest_1samp(final['target_60mins'], 0.0)

    if t_stat > 0:
        one_tailed_p_value = p_value / 2
    else:
        one_tailed_p_value = 1 - (p_value / 2)

    print(f"{test} P-value: {np.round(one_tailed_p_value, 2)} {np.round(t_statistic, 2)}")
print("US500", len(final))

0.0 P-value: 0.0 22.62
0.001 P-value: 0.0 20.59
0.002 P-value: 0.0 18.56
0.003 P-value: 0.0 16.53
0.004 P-value: 0.0 14.5
0.005 P-value: 0.0 12.47
0.006 P-value: 0.0 10.44
0.007 P-value: 0.0 8.41
0.008 P-value: 0.0 6.38
0.009 P-value: 0.0 4.35
0.01 P-value: 0.01 2.32
0.011 P-value: 0.39 0.29
0.012 P-value: 0.96 -1.74
0.013 P-value: 1.0 -3.77
0.014 P-value: 1.0 -5.8
US500 1091


In [15]:
import numpy as np
import pandas as pd
from scipy import stats
# to test if return after 60 mins are greater than {test}
for test in range(0, 15, 1):
    test /= 1000
    final = pd.concat(dfs)
    final = final[final['target_60mins'] != 0.0]
    final['target_60mins'] = final['target_60mins'] - test
    mean = final['target_60mins'].mean()
    n = len(final)
    s = final['target_60mins'].std()

    t_statistic = (mean - 0.0) / (s / np.sqrt(n))

    t_stat, p_value = stats.ttest_1samp(final['target_60mins'], 0.0)

    if t_stat > 0:
        one_tailed_p_value = p_value / 2
    else:
        one_tailed_p_value = 1 - (p_value / 2)

    print(f"{test} P-value: {np.round(one_tailed_p_value, 2)} {np.round(t_statistic, 2)}")
print("US500", len(final))

0.0 P-value: 0.0 22.62
0.001 P-value: 0.0 20.59
0.002 P-value: 0.0 18.56
0.003 P-value: 0.0 16.53
0.004 P-value: 0.0 14.5
0.005 P-value: 0.0 12.47
0.006 P-value: 0.0 10.44
0.007 P-value: 0.0 8.41
0.008 P-value: 0.0 6.38
0.009 P-value: 0.0 4.35
0.01 P-value: 0.01 2.32
0.011 P-value: 0.39 0.29
0.012 P-value: 0.96 -1.74
0.013 P-value: 1.0 -3.77
0.014 P-value: 1.0 -5.8
US500 1091
